In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

load_dotenv(dotenv_path="/home/bipin/Documents/genai/g25-nov-hindi/langchain-learning/.env")

True

In [16]:
model = ChatOpenAI(
        model="gpt-4.1-nano",
        temperature=0.7,
        max_tokens=700,
    )

##  Prompting Techniques

### Zero-shot Prompting

**Definition:** Direct instruction without any examples.

**Importance:** Tests the model's inherent knowledge and capabilities.

**Use Cases:** Simple tasks, general knowledge queries, when examples aren't available.

In [4]:
# Zero-shot prompting example
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a sentiment analysis expert."),
    ("human", "Classify the sentiment of this text as positive, negative, or neutral: {text}")
])

chain = prompt | model | StrOutputParser()

# Test with different texts
texts = [
    "I absolutely love this product! It's amazing!",
    "This is the worst experience I've ever had.",
    "The weather is cloudy today."
]

results = [chain.invoke({"text": text}) for text in texts]
print(results)


['Positive', 'Negative', 'The sentiment of the text "The weather is cloudy today." is neutral.']


In [8]:
# Zero-shot prompting example
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a  amazon product sentiment analysis expert."),
    ("human", "Classify the sentiment from 1 star to 5 star rating: {text}")
])

chain = prompt | model | StrOutputParser()

# Test with different texts
texts = """
    Is quality mein best jute Hain ₹1000 mein aur kya chahie"
"""

results = chain.invoke({"text": texts}) 
print(results)


The sentiment of the review is positive, likely a 4-star or 5-star rating, as it expresses satisfaction with the quality of the shoes for the price.


### One-shot Prompting

**Definition:** Providing a single example to guide the model.

**Importance:** Demonstrates desired format/style with minimal context.

**Use Cases:** Format specification, style guidance, simple pattern recognition.

In [9]:
# One-shot prompting example
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an email classifier. Classify emails as 'spam' or 'not spam'."),
    ("human", "Email: Get rich quick! Click here now!"),
    ("ai", "Classification: spam"),
    ("human", "Email: {email}")
  
])

chain = prompt | model | StrOutputParser()

email = """Your credit card has been pre-approved for a limit increase of ₹5,00,000 with zero annual charges and instant activation.

✅ No income proof required
✅ 0% interest for first 60 days
✅ Free reward points on activation

To claim your upgraded credit card, please verify your details immediately by clicking the link below:

👉 Click here to activate now

⚠️ Note: This offer is valid for 24 hours only. Failure to respond may result in cancellation of your benefits.

Thank you for choosing our services.

Warm regards,
Customer Support Team
Credit Card Services"""
result = chain.invoke({"email": email})

print(f"Email: {email}")
print(f"Result: {result}")

Email: Your credit card has been pre-approved for a limit increase of ₹5,00,000 with zero annual charges and instant activation.

✅ No income proof required
✅ 0% interest for first 60 days
✅ Free reward points on activation

To claim your upgraded credit card, please verify your details immediately by clicking the link below:

👉 Click here to activate now

⚠️ Note: This offer is valid for 24 hours only. Failure to respond may result in cancellation of your benefits.

Thank you for choosing our services.

Warm regards,
Customer Support Team
Credit Card Services
Result: not spam


### Few-shot Prompting

**Definition:** Providing 2-5 examples to establish patterns.

**Importance:** Significantly improves accuracy for specific tasks without fine-tuning.

**Use Cases:** Classification, data extraction, consistent formatting, domain-specific tasks.

In [11]:
# Few-shot prompting example
few_shot_prompt = """
     "The movie was fantastic! I loved every minute.", "output": "positive",
     "Terrible service, will never come back.", "output": "negative",
    "The product works as expected.", "output": "neutral",
"""

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a sentiment classifier. Classify text as positive, negative, or neutral."),
    few_shot_prompt,
    ("human", "{input}"),
])

chain = final_prompt | model | StrOutputParser()

# Test with new input
new_text = "idont like the food, nothing special."
result = chain.invoke({"input": new_text})

print(f"Text: {new_text}")
print(f"Sentiment: {result}")

Text: idont like the food, nothing special.
Sentiment: negative


### Chain-of-Thought (CoT) Prompting

**Definition:** Encouraging step-by-step reasoning.

**Importance:** Dramatically improves performance on complex reasoning tasks.

**Use Cases:** Math problems, logical reasoning, multi-step analysis.

In [12]:
# Chain-of-Thought prompting example
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a math tutor. Solve problems step by step."),
    ("human", """Solve this problem step by step:
    
A store has 45 apples. They sell 18 apples in the morning and 12 apples in the afternoon. 
Then they receive a delivery of 30 more apples. How many apples does the store have now?

Let's think step by step:""")
])

chain = prompt | model | StrOutputParser()
result = chain.invoke({})

print(result)

Let's solve this problem step by step.

**Step 1:** Start with the initial number of apples the store has.
- Initial apples = 45

**Step 2:** Subtract the apples sold in the morning.
- Apples sold in the morning = 18
- Remaining apples after morning sales = 45 - 18 = 27

**Step 3:** Subtract the apples sold in the afternoon.
- Apples sold in the afternoon = 12
- Remaining apples after afternoon sales = 27 - 12 = 15

**Step 4:** Add the new delivery of apples.
- Apples received in delivery = 30
- Apples after delivery = 15 + 30 = 45

**Final answer:** The store has **45 apples** now.


In [14]:
# Chain-of-Thought prompting example
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a math tutor"),
    ("human", """Solve this problem:
    
A store has 45 apples. They sell 18 apples in the morning and 12 apples in the afternoon. 
Then they receive a delivery of 30 more apples. How many apples does the store have now?

""")
])

chain = prompt | model | StrOutputParser()
result = chain.invoke({})

print(result)

Let's break down the problem step by step:

1. The store starts with 45 apples.
2. They sell 18 apples in the morning:  
   45 - 18 = 27 apples remaining.
3. They sell 12 apples in the afternoon:  
   27 - 12 = 15 apples remaining.
4. They receive a delivery of 30 more apples:  
   15 + 30 = 45 apples.

**Answer:** The store now has **45 apples**.


In [17]:
# Chain of Thought prompting example
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a technology analyst."),
    ("human", """Problem:
What will be the impact of AI on IT jobs in the next 10 years?

Think step by step:
1. Identify benefits of AI
2. Identify drawbacks of AI
3. Consider historical examples of automation
4. Draw a final conclusion""")
])

chain = prompt | model | StrOutputParser()
result = chain.invoke({})
print(result)


Certainly! Let’s analyze the potential impact of AI on IT jobs over the next decade by systematically considering each step.

### 1. Benefits of AI
- **Increased Efficiency and Automation:** AI can automate repetitive and mundane tasks such as code testing, system monitoring, and troubleshooting, allowing IT professionals to focus on more strategic initiatives.
- **Enhanced Decision-Making:** AI-driven analytics can provide insights that improve infrastructure management, security, and resource allocation.
- **Innovation Enablement:** AI tools can accelerate software development, facilitate the creation of new products, and enable more sophisticated cybersecurity measures.
- **Job Transformation:** Instead of replacing roles, AI can augment IT professionals’ capabilities, leading to more skilled and specialized positions.

### 2. Drawbacks of AI
- **Job Displacement:** Certain routine or manual IT tasks may become automated, potentially reducing demand for entry-level or administrative

### Tree-of-Thought (ToT) Prompting

**Definition:** Exploring multiple reasoning paths simultaneously.

**Importance:** Enables exploration of different solution strategies.

**Use Cases:** Complex problem-solving, creative tasks, strategic planning.

In [18]:
# Tree-of-Thought prompting example
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a strategic problem solver."),
    ("human", """Problem: How can a small coffee shop increase revenue?

Generate 3 different solution approaches:
1. Approach A: Focus on...
2. Approach B: Focus on...
3. Approach C: Focus on...

Then evaluate each approach and recommend the best one.""")
])

chain = prompt | model | StrOutputParser()
result = chain.invoke({})

print(result)

Certainly! Here are three different solution approaches to help a small coffee shop increase revenue:

**Approach A: Focus on Enhancing Customer Experience and Loyalty**
- Implement loyalty programs (e.g., punch cards, digital rewards)
- Offer exceptional customer service and personalized experiences
- Create a cozy, inviting ambiance to encourage longer visits and repeat business
- Host community events or workshops to build a loyal customer base

**Approach B: Focus on Diversifying Revenue Streams**
- Introduce new product lines like baked goods, sandwiches, or specialty drinks
- Offer merchandise such as branded mugs, beans, or brewing equipment
- Develop catering services for local events or businesses
- Explore partnerships with local vendors for cross-promotions

**Approach C: Focus on Marketing and Online Presence**
- Increase social media marketing and local advertising
- Optimize online ordering and delivery options
- Create a website with an engaging story and menu
- Leverage

In [ ]:
# Tree of Thought prompting example
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a technology foresight and reasoning expert."),
    ("human", """Problem:
What will be the impact of robotics on society in the next 10 years?

Use a Tree of Thought approach:
1. Explore the Positive impact branch
2. Explore the Negative impact branch
3. Explore the Neutral/Mixed impact branch

For each branch:
- Provide key reasoning points
- Briefly evaluate the strength of that branch

Finally:
- Compare all branches
- Select the most reasonable overall conclusion
- Clearly state whether the impact is Positive, Negative, or Neutral.""")
])


### Self-Consistency Prompting

**Definition:** Generating multiple reasoning paths and selecting the most consistent answer.

**Importance:** Improves reliability and reduces errors.

**Use Cases:** Critical decisions, fact verification, complex reasoning.

In [19]:
# Self-consistency prompting example
# Self-consistency prompting for future impact analysis
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a technology impact analyst."),
    ("human", """Problem:
What will be the impact of robotics on jobs in the next 10 years?

Analyze this using 5 different reasoning perspectives 
(economic, social, technological, ethical, and historical).

For each perspective:
1. Give a short reasoning
2. Classify the impact as Positive, Negative, or Neutral

Finally, determine the most consistent overall classification 
based on majority reasoning.""")
])

chain = prompt | model | StrOutputParser()
result = chain.invoke({})

print(result)

Certainly! Here's an analysis of the potential impact of robotics on jobs over the next decade from five different perspectives:

### 1. Economic Perspective
**Reasoning:**  
Robotics can significantly increase productivity and reduce costs for businesses, leading to economic growth. However, automation may displace certain types of jobs, especially repetitive and manual roles, causing short-term unemployment in some sectors. The new wealth generated could eventually create new industries and job opportunities, but there may be a lag in workforce adaptation.

**Impact Classification:**  
**Mixed, leaning toward Positive** — long-term economic growth and efficiency gains, despite short-term displacement.

---

### 2. Social Perspective
**Reasoning:**  
Robotics could widen socio-economic inequalities if displaced workers lack access to retraining or new job opportunities. Conversely, automation can improve working conditions by taking over dangerous or monotonous tasks, potentially enha

### ReAct Prompting (Reasoning + Acting)

**Definition:** Combining reasoning with tool/action execution.

**Importance:** Enables agents to interact with external tools and APIs.

**Use Cases:** Web search, database queries, API calls, multi-step workflows.

In [ ]:
# ReAct prompting example (simulated)
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an AI assistant that uses the ReAct (Reasoning + Acting) framework.
For each task, follow this pattern:
1. Thought: Reason about what to do
2. Action: Specify what action to take
3. Observation: What you would observe from that action
4. Repeat until you can provide a final answer"""),
    ("human", "Task: Find the current population of Tokyo and compare it to New York City.")
])

chain = prompt | model | StrOutputParser()
result = chain.invoke({})

print(result)

### Role-based Prompting

**Definition:** Assigning specific personas or expertise to the model.

**Importance:** Tailors responses to specific domains and audiences.

**Use Cases:** Expert advice, creative writing, technical documentation.

In [ ]:
# Role-based prompting example
roles = [
    "You are a senior Python developer with 10 years of experience.",
    "You are a creative writer specializing in science fiction.",
    "You are a financial advisor helping clients with retirement planning."
]

questions = [
    "What are the best practices for error handling?",
    "Write a short story opening about time travel.",
    "How should I diversify my investment portfolio?"
]

for role, question in zip(roles, questions):
    prompt = ChatPromptTemplate.from_messages([
        ("system", role),
        ("human", question)
    ])
    
    chain = prompt | model | StrOutputParser()
    result = chain.invoke({})
    
    print(f"Role: {role}")
    print(f"Question: {question}")
    print(f"Response: {result[:200]}...\n")
    print("-" * 80 + "\n")